Quick code to join results from different sweeps in just one csv file

In [1]:
import pandas as pd
import numpy as np
import glob, os
import ast

pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)


In [2]:
# Path containing the csv for each sweep done
list_path =['/Users/wola/Documents/MSS/Personales/GitRepos/PFM_EconomicNewsImpact/data/models_results']

# We concatenate the results of each sweep in a unique dataframe
all_files = [glob.glob(os.path.join(path, 'sweeps_*.csv')) for path in list_path]             
df_from_each_file = (pd.read_csv(file) for list_files in all_files for file in list_files )
df = pd.concat(df_from_each_file, ignore_index=True)


In [3]:
df.head(3)

,Unnamed: 0,model_type,model,direction,sweeps_market_variables,sweep_news_agg,sweep_buy_sell,before_data,sweep_grid,best_score,best_params,f1_microavg,precision_weighted,precision_below_TH,recall_below_TH,precision_above_TH,recall_above_TH,support_below_TH,support_above_TH,report,elapsed_time
0,0,clf,kn,max,basic,ALL_NO_1_1_1,0_120,included,basic,0.75,{'n_neighbors': 71},0.73,0.67,0.73,1.00,0.5,0.00,947.0,345.0,"{'0': {'precision': 0.7333333333333333, 'recal...",0.57
1,1,clf,svc-rbf,max,basic,ALL_NO_1_1_1,0_120,included,basic,0.75,"{'C': 1, 'gamma': 1}",0.73,0.54,0.73,1.00,0.0,0.00,947.0,345.0,"{'0': {'precision': 0.7329721362229102, 'recal...",7.81
2,2,clf,dtree,max,basic,ALL_NO_1_1_1,0_120,included,basic,0.75,"{'max_depth': 4, 'min_samples_leaf': 250}",0.73,0.68,0.74,0.97,0.5,0.09,947.0,345.0,"{'0': {'precision': 0.7443181818181818, 'recal...",0.49


In [4]:
len(df)

3192

In [5]:
df.to_csv('/Users/wola/Documents/MSS/Personales/GitRepos/PFM_EconomicNewsImpact/data/models_results/all_sweeps.csv')

In [52]:
df_agg = pd.read_csv('./models_out/ALL_YES_3_2_1-basic-included-60_120-basic.csv')

In [53]:
df_agg.columns

Index(['Unnamed: 0', 'datetime', 'new_id', 'forecast_error_diff_deviation', 'forecast_error_diff_outlier_class', 'previous_error_diff_deviation', 'previous_error_diff_outlier_class', 'fe_accurate', 'fe_better', 'fe_worse', 'pe_accurate', 'pe_better', 'pe_worse', 'High', 'Low', 'Medium', 'year', 'week', 'weekday', 'num_news', 'volatility_0_5_after', 'pips_agg_0_5_after', 'volatility_5_10_after', 'pips_agg_5_10_after', 'volatility_10_15_after', 'pips_agg_10_15_after', 'volatility_15_20_after', 'pips_agg_15_20_after', 'volatility_20_25_after', 'pips_agg_20_25_after', 'volatility_25_30_after', 'pips_agg_25_30_after', 'volatility_0_30_after', 'pips_agg_0_30_after', 'volatility_0_60_after', 'pips_agg_0_60_after', 'volatility_60_0_before', 'pips_agg_60_0_before', 'pips_agg_0_120_after', 'direction_agg_0_120_after'], dtype='object')

In [90]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report


columns_of_model = ['new_id', 'forecast_error_diff_deviation', 'forecast_error_diff_outlier_class', 
                    'previous_error_diff_deviation', 'previous_error_diff_outlier_class', 'fe_accurate', 
                    'fe_better', 'fe_worse', 'pe_accurate', 'pe_better', 'pe_worse', 'High', 'Low', 'Medium',
                    'year', 'week', 'weekday', 'num_news', 'volatility_0_5_after', 'pips_agg_0_5_after', 
                    'volatility_5_10_after', 'pips_agg_5_10_after', 'volatility_10_15_after', 'pips_agg_10_15_after', 
                    'volatility_15_20_after', 'pips_agg_15_20_after', 'volatility_20_25_after', 'pips_agg_20_25_after',
                    'volatility_25_30_after', 'pips_agg_25_30_after', 'volatility_0_30_after', 'pips_agg_0_30_after', 
                    'volatility_0_60_after', 'pips_agg_0_60_after', 'volatility_60_0_before', 'pips_agg_60_0_before']

X = df_agg[columns_of_model].values
y = df_agg['direction_agg_0_120_after'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [91]:
clf_dt = DecisionTreeClassifier(min_samples_leaf= 10, max_depth= 3)

In [92]:
clf_rf = RandomForestClassifier(max_depth= 7, min_samples_leaf= 10, n_estimators= 200)

In [93]:
clf_vot = VotingClassifier(estimators=[('dt', clf_dt),('rf', clf_rf)])

In [94]:
clf_vot.fit(X_train, y_train)

VotingClassifier(estimators=[('dt', DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=10, min_samples_split=2,
            min_weight_fraction_leaf=0...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))],
         flatten_transform=None, n_jobs=None, voting='hard', weights=None)

In [95]:
y_predict = clf_vot.predict(X_test)
classification_report(y_test, y_predict, output_dict=True)

{'0': {'precision': 0.6386138613861386,
  'recall': 0.6292682926829268,
  'f1-score': 0.6339066339066339,
  'support': 205},
 '1': {'precision': 0.6951219512195121,
  'recall': 0.8109756097560976,
  'f1-score': 0.74859287054409,
  'support': 492},
 '2': {'precision': 0.7446808510638298,
  'recall': 0.4772727272727273,
  'f1-score': 0.5817174515235458,
  'support': 220},
 'micro avg': {'precision': 0.6902944383860414,
  'recall': 0.6902944383860414,
  'f1-score': 0.6902944383860414,
  'support': 917},
 'macro avg': {'precision': 0.6928055545564935,
  'recall': 0.6391722099039173,
  'f1-score': 0.6547389853247566,
  'support': 917},
 'weighted avg': {'precision': 0.6943790935858244,
  'recall': 0.6902944383860414,
  'f1-score': 0.682918638597309,
  'support': 917}}

Ok, this ensembled model is not better than the two individual ones